In [5]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys

In [6]:
df = pd.read_csv('Mcule_10000.csv', index_col=0)

In [ ]:
sys.path.append(os.path.dirname(os.path.abspath('feature_utils.py')))
from feature_utils import get_clean_res_list

In [7]:
def extract_protein_names(file_path):
    protein_names = []
    with open(file_path, 'r') as file:
        for line in file:
            # Get the base name without directory path
            base_name = os.path.basename(line.strip())
            # Split the base name to get the protein name without extension
            protein_name = os.path.splitext(base_name)[0]
            protein_names.append(protein_name)
    return protein_names

file_path = 'pdb_dataset.ds'
protein_names = extract_protein_names(file_path)
print(protein_names[0:5])

['1A07', '1A08', '1A09', '1A0N', '1A1A']


In [9]:
from Bio.PDB import PDBParser
from feature_utils import get_clean_res_list, get_protein_feature # install gvp.data via git on laptop

def process_proteins(protein_names, pdb_directory):
    parser = PDBParser(QUIET=True)
    protein_dict = {}

    for proteinName in protein_names:
        proteinFile = f"{pdb_directory}/{proteinName}.pdb"
        s = parser.get_structure(proteinName, proteinFile)
        res_list = list(s.get_residues())
        clean_res_list = get_clean_res_list(res_list, ensure_ca_exist=True)
        protein_dict[proteinName] = get_protein_feature(clean_res_list)
    
    return protein_dict

pdb_directory = "PDB_files"  # Directory containing PDB files

protein_dict = process_proteins(protein_names, pdb_directory)

ModuleNotFoundError: No module named 'gvp.data'

In [ ]:
info = []
for protein_name in protein_names:
    for i, line in tqdm(df.iterrows(), total=len(df)):
        smiles = line['smiles']
        compund_name = ''
        protein_name = protein_name
        com = ",".join([str(x.round(3)) for x in protein_dict[protein_name][0].mean(axis=0).numpy()])
        info.append([protein_name, compund_name, smiles, "protein_center", com])

info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'smiles', 'pocket_name', 'pocket_com'])


In [10]:
import torch
torch.set_num_threads(1)

from torch_geometric.data import Dataset
# from utils import construct_data_from_graph_gvp
import rdkit.Chem as Chem    # conda install rdkit -c rdkit if import failure.
# from feature_utils import extract_torchdrug_feature_from_mol, get_canonical_smiles